# 검증 세트

- 테스트 세트를 사용해 모델의 성능을 개선해나가다보면 점점 테스트 세트에 적합한 모델이 됨 
    - 테스트 세트를 통해 일반화 성능을 올바르게 예측하기 힘들어짐 
    
- 따라서 테스트 세트를 사용하지 않고 훈련 세트를 또 다시 나눠 validaton_set를 이용 
- 검증 세트 활용
    1. 훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가
    2. 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 선택
    3. 최적의 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련
    4. 테스트 세트에서 최종 점수를 평가 

In [34]:
import pandas as pd

# train_test_split, cross_validate, StratifiedGroupKFold, 교차검증 
# 하이퍼파라미터 기능 : GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import (train_test_split, cross_validate, StratifiedKFold,
                                      GridSearchCV, RandomizedSearchCV)

from sklearn.tree import DecisionTreeClassifier
import numpy as np
from scipy.stats import uniform, randint

In [18]:
df = pd.read_csv("./data/wine.csv")

In [19]:
df.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [20]:
x = df.drop('class', axis = 1)
y = df['class']

- 보다가 이해 안가면 바로 그림 그리기 

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y,
                                                    random_state = 11)

In [27]:
# train 데이터 한번 더 나누기
x_sub, x_val, y_sub, y_val = train_test_split(x_train, y_train, test_size = 0.2,
                                              stratify = y_train, random_state = 11)

In [28]:
print(x_sub.shape, x_val.shape, x_test.shape)

(4157, 3) (1040, 3) (1300, 3)


In [29]:
print(x_train.shape, x_test.shape)

(5197, 3) (1300, 3)


## 모델 훈련

In [30]:
# 훈련 99, 검증 데이터 86 
dt = DecisionTreeClassifier(random_state = 11)
dt.fit(x_sub, y_sub)
print(dt.score(x_sub, y_sub))
print(dt.score(x_val, y_val))

0.9980755352417608
0.8663461538461539


- 훈련 세트 점수가 지나치게 높고 검증 세트 점수와의 차이가 커서 과대적합 되었을 가능성이 높음

# 교차 검증

<img src = "./image/kfold.png">

- 검증 세트를 만드는 과정에서 훈련 세트가 줄어들었음 
    - 일반적으로 많은 데이터를 훈련에 사용할 수록 좋은 모델이 만들어질 가능성이 높음 
        - 검증 세트를 줄이면 검증 점수가 불안정해져 올바른 모델 검증이 힘듦
        - 검증 세트를 늘리면 훈련 세트가 줄어듦
- 따라서 cross validation(교차 검증)을 이용하여 안정적인 검증 점수를 얻으면서 훈련에 더 많은 데이터를 사용할 수 있음 

In [31]:
# cross_validate 교차검증 함수
# cross_validate(모델 객체, 훈련 세트 전체 독립변수, 훈련 세트 전체 종속변수)
scores = cross_validate(dt, x_train, y_train)
print(scores)

{'fit_time': array([0.00699854, 0.00454402, 0.        , 0.02101302, 0.00597095]), 'score_time': array([0.00200105, 0.        , 0.        , 0.00100422, 0.00203228]), 'test_score': array([0.83557692, 0.85      , 0.84985563, 0.86429259, 0.84408085])}


- fit_time : 모델을 훈련하는 시간
- score_time : 모델을 검증하는 시간
- test_score : 검증 점수
    - 교차 검증의 최종 점수 : test_score의 점수 평균 

#### 주의할 점

- cross_validate는 훈련 세트를 섞지 않음
    - 지금의 예제에서는 train_test_split 으로 전체 데이터를 섞었기 때문에 따로 섞을 필요는 없음
    - 교차 검증시에 훈련 세트를 섞어야 한다면 splitter(분할기)를 지정해야 함 
    
- 분할기
    - 교차 검증에서 폴드를 어떻게 나눌지 결정함
    - cross_validate 함수는 기본적으로 회귀모델일 경우 KFold를 사용하고, 분류 모델일 경우 종속 변수의 범주를 골고루 나누기 위해 StratifiedKFold를 사용함 

In [35]:
# StratifiedKFold 10-폴드 교차 검증 
# 보통은 n_splits 4~10, 강사님은 4,5 선호 
splitter = StratifiedKFold(n_splits= 10, shuffle = True, random_state = 11)
scores = cross_validate(dt, x_train, y_train, cv = splitter)
print(np.mean(scores['test_score']))

0.8630005928560841


# 그리드 서치

- 머신러닝 모델이 학습하는 파라미터 : 모델 파라미터 
- 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터 : 하이퍼파라미터 
- 하이퍼파라미터 튜닝 순서
    1. 기본값으로 모델 훈련 
    2. 검증 세트의 점수나 교차 검증을 통해서 매개변수를 수정 
        - 모델마다 적게는 1 ~2개, 많게는 5 ~ 6개 정도의 매개변수를 제공함
        
- 하이퍼파라미터 튜닝을 할 때에는 여러 매개변수의 최적값을 동시에 찾아야 함
    - 예) A 매개변수의 최적값을 찾았다고 하더라도 B매개변수의 값이 바뀌면 A매개변수의 최적값이 바뀜
    - 매개변수가 많아질수록 최적값을 찾는 과정이 더 복잡해짐 
    
- 위의 문제를 해결하기 위해서 사이킷런의 GridSearch(그리드 서치)를 사용
    - 하이퍼파라미터 탐색과 교차 검증을 한번에 수행 

In [36]:
params = {"min_impurity_decrease" : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [37]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 11), params)

- 그리드 서치는 입력된 모델의 파라미터 값을 바꿔가면서 훈련을 실행함
    - 현재 예제에서는 min_impurity_decrease를 바꿔가면서 5회 훈련을 실행함 
- GridSearchCV의 cv매개변수 기본값은 5이기 때문에 5-폴드 교차 검증을 수행함 
- 따라서 1회 훈련마다 5회 교차 검증 * 5회 훈련 = 25회 모델을 훈련 

In [38]:
gs.fit(x_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=11),
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

- 그리드 서치는 훈련이 끝나면 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련함
    - 해당 모델은 best_estimator_ 속성에 저장됨 

In [39]:
dt = gs.best_estimator_
print(dt.score(x_train, y_train))

0.9012892053107562


- 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장됨 

In [40]:
gs.best_params_

{'min_impurity_decrease': 0.0003}

- 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 "mean_test_score" 키에 저장됨

In [41]:
gs.cv_results_

{'mean_fit_time': array([0.00647736, 0.00460095, 0.00311947, 0.00455604, 0.00313158]),
 'std_fit_time': array([0.00045504, 0.0013161 , 0.00623894, 0.00578229, 0.00626316]),
 'mean_score_time': array([0.0017982 , 0.0012084 , 0.        , 0.00144539, 0.00309796]),
 'std_score_time': array([0.00074083, 0.0007562 , 0.        , 0.00204371, 0.00619593]),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_impurity_decrease': 0.0001},
  {'min_impurity_decrease': 0.0002},
  {'min_impurity_decrease': 0.0003},
  {'min_impurity_decrease': 0.0004},
  {'min_impurity_decrease': 0.0005}],
 'split0_test_score': array([0.84615385, 0.85961538, 0.85480769, 0.86057692, 0.84711538]),
 'split1_test_score': array([0.85769231, 0.85961538, 0.85576923, 0.85192308, 0.84615385]),
 'split2_test_score': array([0.85466795, 0.86814244, 0.87006737, 0.86814

In [43]:
gs.cv_results_['mean_test_score']

array([0.85433997, 0.8626142 , 0.86396332, 0.86280799, 0.85819186])

In [44]:
best_idx = np.argmax(gs.cv_results_['mean_test_score'])
best_idx

2

In [45]:
# 어느 파라미터가 최고의 점수가 나왔는지도 나옴 
gs.cv_results_['params'][best_idx]

{'min_impurity_decrease': 0.0003}

#### min_impurity_decrease, max_depth, min_samples_split의 최적값 찾기

- min_impurity_decrease : 노드를 분할하기 위한 불순도 감소 최소량
- max_depth : 트리의 깊이 제한
- min_samples_split : 노드를 나누기 위한 최소 샘플 수 

In [47]:
params = {"min_impurity_decrease" : np.arange(0.0001, 0.001, 0.0001), # 총 9개 
          "max_depth" : range(5, 20), # 총 15개
          "min_samples_split" : range(2, 100, 10)} # 총 10개 ( 2부터 100까지 10씩 올림)

- 위의 파라미터로 수행할 교차 검증 횟수는 9 * 15 * 10 = 1350회
- 기본 5-폴드 교차 검증을 하기 때문에 1350 * 5 = 6750번의 훈련을 수행 

In [48]:
# 교차검증이니까 마지막에 5 곱해주기
9 * 15 * 10 *5

1350

In [52]:
# n_jobs : 병렬수행: - 지정해주면 모든 코어를 다 사용하기때문에 컴퓨터는 뜨거워지겠지만, 속도는 많이 빨라집니다.
gs = GridSearchCV(DecisionTreeClassifier(random_state = 11), params, n_jobs = -1)
gs.fit(x_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=11), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [54]:
# 최상의 매개변수 조합
gs.best_params_

{'max_depth': 7,
 'min_impurity_decrease': 0.0007000000000000001,
 'min_samples_split': 82}

In [53]:
# 최상의 교차 검증 검수
np.max(gs.cv_results_["mean_test_score"])

0.8655004812319538

# 랜덤 서치

- 매개변수의 값이 수치일 때 값의 범위나 간격을 정하기 어려울 수 있고 너무 많은 매개변수 조건이 있어서 그리드 서치 수행 시간이 오래 걸릴 수 있음
- RandomSearch(랜덤 서치)는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달함

- 균등 분포에서 샘플링
    - 주어진 범위에서 고르게 값을 생성
    - randint : 정수
    - uniform : 실수 

In [55]:
# 0 ~ 10 사이의 랜덤한 객체 
randint(0, 10).rvs(10)

array([7, 2, 6, 9, 9, 7, 1, 5, 2, 1], dtype=int64)

In [56]:
uniform(0, 1).rvs(10)

array([0.97044293, 0.79873958, 0.59384066, 0.19551584, 0.12599238,
       0.86512729, 0.16667684, 0.96086457, 0.68924536, 0.67058865])

In [57]:
# 랜덤하게 값을 뽑아서 파라미터로 값을 설정할 것임 
params = {"min_impurity_decrease" : uniform(0.0001, 0.001),
          "max_depth" : randint(20, 50),
          "min_samples_split" : randint(2, 25),
          "min_samples_leaf" : randint(1, 25)}

In [58]:
#  n_iter : 몇번 랜덤하게 뽑을건지 
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 11), params, n_iter = 100,
                        n_jobs = -1, random_state = 11)
rs.fit(x_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=11),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001815EF42F90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001815EC04D50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001815EC81F10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001815EF43490>},
                   random_state=11)

- 위 params에 정의된 매개변수 범위에서 총 100번 샘플링하여 교차 검증을 수행 
    - 그리드 서치보다 교차 검증 수를 줄이면서도 넓은 영역을 효과적으로 탐색 

In [59]:
print(rs.best_params_)

{'max_depth': 36, 'min_impurity_decrease': 0.0010841126580515485, 'min_samples_leaf': 22, 'min_samples_split': 17}


In [60]:
np.max(rs.cv_results_['mean_test_score'])

0.8639625749611313

In [61]:
print(gs.best_estimator_.score(x_test, y_test))
print(rs.best_estimator_.score(x_test, y_test))

0.8730769230769231
0.87
